In [22]:
# ------------------------------
# Step 1: Import Libraries
# ------------------------------
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv(r"D:\Projects\Capstone Project\Loan_Prediction.csv")

# Preview dataset
print("Shape of dataset:", df.shape)
print("------------------------------")
print(df.head())
print("------------------------------")
print(df.info())
print("------------------------------")
print(df.describe())


Shape of dataset: (614, 13)
------------------------------
    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1      

In [23]:
# ------------------------------
# Step 2: Handle Missing Values
# ------------------------------

# Check missing values
print("\nMissing values per column:\n", df.isnull().sum())

# Fill numerical columns with median
df['LoanAmount'].fillna(df['LoanAmount'].median(), inplace=True)
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].median(), inplace=True)
df['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)

# Fill categorical columns with mode
for col in ['Gender','Married','Dependents','Self_Employed']:
    df[col].fillna(df[col].mode()[0], inplace=True)



Missing values per column:
 Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


C:\Users\kumar\AppData\Local\Temp\ipykernel_6164\3101133064.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['LoanAmount'].fillna(df['LoanAmount'].median(), inplace=True)
C:\Users\kumar\AppData\Local\Temp\ipykernel_6164\3101133064.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

In [24]:
# ------------------------------
# Step 3: Standardize Data Types
# ------------------------------

# Convert Dependents column: replace '3+' with 3
df['Dependents'] = df['Dependents'].replace('3+', 3).astype(int)

# Verify types
print(df.dtypes)


Loan_ID               object
Gender                object
Married               object
Dependents             int64
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object


In [25]:
# ------------------------------
# Step 4: Feature Engineering
# ------------------------------

# Total Income
df['Total_Income'] = df['ApplicantIncome'] + df['CoapplicantIncome']

# Income Bracket (low < 4000, medium 4000-8000, high > 8000)
df['Income_Bracket'] = pd.cut(df['Total_Income'],
                              bins=[0,4000,8000,20000],
                              labels=['Low','Medium','High'])

# Loan to Income Ratio
df['Loan_to_Income_Ratio'] = df['LoanAmount'] / df['Total_Income']

# Credit history categorical
df['Credit_History_Flag'] = df['Credit_History'].map({1.0: 'Good', 0.0: 'Bad'})


In [26]:
# ------------------------------
# Step 5: Encode Categorical Features
# ------------------------------

# Map Loan_Status to numeric
df['Loan_Status_Binary'] = df['Loan_Status'].map({'Y':1, 'N':0})

# Quick check of final cleaned data

print(df.head())
print("------------------------------------------------------------ \n------------------------------------------------------------")

print(df.info())



    Loan_ID Gender Married  Dependents     Education Self_Employed  \
0  LP001002   Male      No           0      Graduate            No   
1  LP001003   Male     Yes           1      Graduate            No   
2  LP001005   Male     Yes           0      Graduate           Yes   
3  LP001006   Male     Yes           0  Not Graduate            No   
4  LP001008   Male      No           0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0       128.0             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  Total_Income Income_Bracket  \
0             1.0         Urban           Y        5849.0         Medium   



------------------------------------------------------------ 
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Loan_ID               614 non-null    object  
 1   Gender                614 non-null    object  
 2   Married               614 non-null    object  
 3   Dependents            614 non-null    int64   
 4   Education             614 non-null    object  
 5   Self_Employed         614 non-null    object  
 6   ApplicantIncome       614 non-null    int64   
 7   CoapplicantIncome     614 non-null    float64 
 8   LoanAmount            614 non-null    float64 
 9   Loan_Amount_Term      614 non-null    float64 
 10  Credit_History        614 non-null    float64 
 11  Property_Area         614 non-null    object  
 12  Loan_Status           614 non-null    

In [29]:
# ------------------------------
# Exploratory Analysis in Python
# ------------------------------


# Distribution of Loan_Status
print(df['Loan_Status'].value_counts(normalize=True) * 100)
print("------------------------------------------------------------")
# Average loan amount by credit history
print(df.groupby('Credit_History')['LoanAmount'].mean())
print("------------------------------------------------------------")
# Loan approval rate by Income Bracket
print(df.groupby('Income_Bracket')['Loan_Status_Binary'].mean() * 100)
print("------------------------------------------------------------")
# Avg Loan_to_Income_Ratio for approved vs rejected
print(df.groupby('Loan_Status_Binary')['Loan_to_Income_Ratio'].mean())
print("------------------------------------------------------------")
# Loan approvals by Property Area
print(df.groupby('Property_Area')['Loan_Status_Binary'].mean() * 100)


Loan_Status
Y    68.729642
N    31.270358
Name: proportion, dtype: float64
------------------------------------------------------------
Credit_History
0.0    145.876404
1.0    145.731429
Name: LoanAmount, dtype: float64
------------------------------------------------------------
Income_Bracket
Low       67.832168
Medium    69.321534
High      71.052632
Name: Loan_Status_Binary, dtype: float64
------------------------------------------------------------
Loan_Status_Binary
0    0.024988
1    0.023371
Name: Loan_to_Income_Ratio, dtype: float64
------------------------------------------------------------
Property_Area
Rural        61.452514
Semiurban    76.824034
Urban        65.841584
Name: Loan_Status_Binary, dtype: float64


C:\Users\kumar\AppData\Local\Temp\ipykernel_6164\3108720351.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('Income_Bracket')['Loan_Status_Binary'].mean() * 100)


In [30]:
df.to_csv("D:\Projects\Capstone Project\cleaned_loans.csv", index=False)
print("✅ Cleaned data saved as cleaned_loans.csv")


✅ Cleaned data saved as cleaned_loans.csv


In [2]:
df_cleaned_loans.to_excel("D:\Projects\Capstone Project\cleaned_loans.xlsx", index=False)


NameError: name 'df_cleaned_loans' is not defined